# Large Scale Face recognition

What if your dataset contains thousands of identities, with thousands of pictures for each of them? It will be impossible to compare their embeddings one-by-one to find the closest match. This notebook will show you how to use `NMSLIB`, a similarity search library based on approximate nearest neighbors (ANN).

source: https://github.com/nmslib/nmslib 